# Aufgabe Beschreibung

- Mindestens eine NoSQL DB verwenden (Docker, Docker-compose)
- Lessons Learned wichtiger als optimale Lösung (Was hätte ich anders gemacht?)

## Abzugeben:
- Dockerfile / Dockercompose pro DB
- PDF Datenmodell -> Aufbau von System
- Skript / Programm zum Laden von Daten in die DB
- Abfragen zum Szenarien
- PDF Lessons Learned (`lessons-learned.pdf`)

## DB:
- Sollte auf mehrere Container / Knoten laufen
  - Wenn es nicht geht, erklären wieso das nicht ging / was dafür gebraucht ist

## App (Optional)
- Laden von Init-Daten
- Abfragen feuern
- Inhalt anzeigen

## Systemanforderungen
- Es gibt:
  - Follower-Beziehungen
  - Posts von Prominenten
- Aufgaben:
  - Posts von Prominenten auf die 100 IDs verteilen, die am meisten Follower haben (Influencer)
  - Posts können geliked werden (von welchem User wurde ein Post eines anderen Users geliked)
    - Zufällig generiert
- Anfragen:
  - Auflistung von zu einem Account zugeordneten Posts
  - Auflistung der 100 Accounts mit den meisten Followern (Influencer)
  - Auflistung der 100 Accounts, die den meisten der Influencer folgen
  - Startseite für ein beliebiges Account (Influencer sind hier gut):
    - Anzahl Followers
    - Anzahl gefolgte Accounts
    - 25 Posts:
      - Neueste
      - Meisten Likes von gefolgte Accounts
    - Caching der Posts für die Startseite:
      - Fan-Out in den Cache jedes Followers beim Schreiben eines neuen Posts (Fragen nicht von zentraler Tabelle, sondern jedes Account hat eigenen Tweets-Feed)
    - Auflistung der 25 Posts, die ein Wort beinhalten:
      - (Optional: Und-verknüpfte Wörter)


# Setup

## Install needed libraries and import components

In [23]:
# Install needed libraries and import components
!%pip install "pymongo[srv]"

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime

zsh:fg:1: no job control in this shell.


## Define the MongoDB server details

In [3]:
# Define the MongoDB server details
host = 'localhost'
port = 27017
username = 'devroot'  # Replace with your MongoDB username
password = 'devroot'  # Replace with your MongoDB password

# Create the connection string
connection_string = f'mongodb://{username}:{password}@{host}:{port}'

## Connect to DB and test connection

In [10]:
# Connect to the MongoDB server
client = MongoClient(connection_string)

In [11]:
try:
    # Verify connection
    client.admin.command('ping')
    print("Connected successfully to MongoDB")
    
    # List all databases
    databases = client.list_database_names()
    print("Databases:", databases)
        
except ConnectionFailure as e:
    print(f"Could not connect to MongoDB: {e}")

Connected successfully to MongoDB
Databases: ['admin', 'config', 'local']


# Insert Data into DB

## Collections definition

In [20]:
# Select the database and collections
db = client['social_network']
users_collection = db['users']
followers_collection = db['followers']
posts_collection = db['posts']
likes_collection = db['likes']
feeds_collection = db['feeds']

## Help Functions

In [27]:
# Function to add a user
def add_user(user_id):
    user = {"_id": user_id, "following_count": 0, "followers_count": 0}
    users_collection.insert_one(user)

# Function to follow a user
def follow_user(follower_id, followed_id):
    relationship = {"follower_id": follower_id, "followed_id": followed_id}
    followers_collection.insert_one(relationship)
    users_collection.update_one({"_id": follower_id}, {"$inc": {"following_count": 1}})
    users_collection.update_one({"_id": followed_id}, {"$inc": {"followers_count": 1}})

# Function to add a post
def add_post(user_id, content):
    post = {
        "user_id": user_id,
        "content": content,
        "timestamp": datetime.now(),
        "likes": 0
    }
    post_id = posts_collection.insert_one(post).inserted_id
    propagate_post_to_followers(post_id, user_id, content)
    return post_id

# Function to propagate a post to all followers' feeds
def propagate_post_to_followers(post_id, user_id, content):
    followers = followers_collection.find({"followed_id": user_id})
    for follower in followers:
        feed_entry = {
            "user_id": follower["follower_id"],
            "post_id": post_id,
            "poster_id": user_id,
            "content": content,
            "timestamp": datetime.utcnow(),
            "likes": 0
        }
        feeds_collection.insert_one(feed_entry)

# Function to like a post
def like_post(user_id, post_id):
    like = {"user_id": user_id, "post_id": post_id}
    likes_collection.insert_one(like)
    posts_collection.update_one({"_id": post_id}, {"$inc": {"likes": 1}})
    feeds_collection.update_many({"post_id": post_id}, {"$inc": {"likes": 1}})


## Data insertion

In [29]:
# Add some users
add_user(1)
add_user(2)
add_user(3)

# Establish some follower relationships
follow_user(1, 2)  # Alice follows Bob
follow_user(3, 2)  # Charlie follows Bob
follow_user(1, 3)  # Alice follows Charlie

# Add some posts
alice_post_id = add_post(1, "Hello, world!")  # Alice's post
bob_post_id = add_post(2, "It's a beautiful day!")  # Bob's post
charlie_post_id = add_post(3, "Learning MongoDB with Python!")  # Charlie's post


# Like some posts
like_post(1, alice_post_id)  
like_post(3, alice_post_id)
like_post(2, charlie_post_id)

# Request data from DB

## Helper functions

In [39]:
def get_top_influencers(n):
    return users_collection.find().sort("followers_count", -1).limit(n)

def get_user_posts(user_id):
    return posts_collection.find({"user_id": user_id})

def get_top_follower_users(n):
    return users_collection.find().sort("following_count", -1).limit(n)

def get_user_profile(user_id, n):
    user = users_collection.find_one({"_id": user_id})
    followers_count = user["followers_count"]
    following_count = user["following_count"]
    
    recent_posts = posts_collection.find({"user_id": user_id}).sort("timestamp", -1).limit(n)
    popular_posts = posts_collection.find({"user_id": user_id}).sort("likes", -1).limit(n)
    feed = feeds_collection.find({"user_id": user_id}).sort("timestamp", -1).limit(n)
    
    profile = {
        "user_id": user_id,
        "followers_count": followers_count,
        "following_count": following_count,
        "recent_posts": list(recent_posts),
        "popular_posts": list(popular_posts),
        "feed": list(feed)
    }
    return profile

def print_user_profile(user_profile):
    print("User profile with id:", user_profile["user_id"])
    print("Followers count:", user_profile["followers_count"])
    print("Following count:", user_profile["following_count"])
    print("Recent posts:")
    for post in user_profile["recent_posts"]:
        print(post["content"], "date:", post["timestamp"])
    print("Popular posts:")
    for post in user_profile["popular_posts"]:
        print(post["content"], "likes", post["likes"])
    print("Feed:")
    for post in user_profile["feed"]:
        print(post["content"], "date:", post["timestamp"])

def get_posts_with_word(word, n):
    return posts_collection.find({"content": {"$regex": word, "$options": "i"}}).limit(n)


## Find out top 100 most followed

In [34]:
# Get and print top influencers
top_influencers = get_top_influencers(2)
print("Top influencers:")
for influencer in top_influencers:
    print("user id:", influencer["_id"], "Follower count", influencer["followers_count"])

Top influencers:
user id: 2 Follower count 2
user id: 3 Follower count 1


## Find out top 100 influencer followers

In [35]:
# Get and print top followers
top_followers = get_top_follower_users(2)
print("Top followers:")
for follower in top_followers:
    print("user id:", follower["_id"], "Influencers followed", follower["following_count"])

Top followers:
user id: 1 Influencers followed 2
user id: 3 Influencers followed 1


## Show user profile

In [41]:
# Get and print user profile
user_profile = get_user_profile(3, 2)
print_user_profile(user_profile)

User profile with id: 3
Followers count: 1
Following count: 1
Recent posts:
Learning MongoDB with Python! date: 2024-06-18 11:02:34.072000
Popular posts:
Learning MongoDB with Python! likes 1
Feed:
It's a beautiful day! date: 2024-06-18 09:02:34.071000


## Posts containing word

In [42]:
# Get and print posts with a given word
posts_with_word = get_posts_with_word("beautiful", 2)
print("Posts containing 'beautiful':")
for post in posts_with_word:
    print(post["content"])

Posts containing 'beautiful':
It's a beautiful day!


# Cleanup

In [28]:
db.users.drop()
db.followers.drop()
db.posts.drop()
db.likes.drop()
db.feeds.drop()
print("Database cleared.")

Database cleared.


In [7]:
# Close the connection
client.close()